# Scaling
In onze statistische analyse verzamelen we jaarlijks meer gegevens doordat steeds meer mensen hun waarnemingen loggen op waarnemingen.be. We kennen het totale aantal waarnemingen van vogels. We gaan ervan uit dat het aandeel van elke vogelsoort ten opzichte van elkaar constant blijft als de populatie constant blijft. Als het aandeel van een bepaalde vogelsoort stijgt, nemen we aan dat er daadwerkelijk meer vogels van die soort voorkomen.

In [ ]:
df_year_count_cleaned.head()

In [ ]:
# calculate scale_factor
observation_count_2024 = df_year_count_cleaned[df_year_count_cleaned['year'] == 2024]['observation_count'].mean()
individu_count_2024 = df_year_count_cleaned[df_year_count_cleaned['year'] == 2024]['individu_count'].mean()

df_year_count_cleaned['scale_factor_observations'] = df_year_count_cleaned.apply(lambda x: observation_count_2024 / x['observation_count'], axis=1)
df_year_count_cleaned['scale_factor_individus'] = df_year_count_cleaned.apply(lambda x: individu_count_2024 / x['individu_count'], axis=1)


# merge with yearly observation count
df_observations_scaled = df_observations_cleaned.copy()
df_observations_scaled['year_observation'] = df_observations_scaled['date'].dt.year
df_observations_scaled = df_observations_scaled.merge(df_year_count_cleaned, how='left', left_on='year_observation', right_on='year')

# # observations and amount in 2024
# observation_count_2024 = df_observations_scaled[df_observations_scaled['year_observation'] == 2024].shape[0]
# observation_amount_2024 = df_observations_scaled[df_observations_scaled['year_observation'] == 2024]['amount'].sum()

# print(f"Observations in 2024: {observation_count_2024}")
# print(f"Amount of observations in 2024: {observation_amount_2024}")

# scales amount of observations and individual observations
df_observations_scaled['observations_scaled'] = df_observations_scaled.apply(lambda x: 1 * x['scale_factor_observations'], axis=1)
df_observations_scaled['amount_scaled'] = df_observations_scaled.apply(lambda x: x['amount'] * x['scale_factor_observations'], axis=1)